# Clean up data, convert to dataframes, and save to h5

The BAC file is pipe separated with the following fields:
LogLevel [TimeStamp]:[LogVisibility][LogSeverity][entryType][entrySubType][eventType][EventType dependent strings]

The information changes at the end of each entry based on its EventType.  Here is a list of the supported event types and the subsequent additional information for each listed below.

GeneralMessage - [string message]

LevelChangedEvent - [load ID][loadName][roomName][rampTime][rampBaseValue][rampFinalValue]

ButtonChangedEvent - [keypad ID][keypadName][roomName][buttonNum][buttonState]

RemoteSystemEvent - [signalID][signalName][roomName][RemoteSystemEvent string]
TimeClockChangedEvent/OccupancyChangedEvent/SceneChangedEvent - [ID][name][roomName][message]

ConnectionStatus - [device ID][Name][room Name][connection status][Load 1 Room Name:Load 1 Name]|[Load 2 Room Name:Load 2 Name]

    NOTE: DeviceConnectionStatusWithOptions is the same format as ConnectionStatus. 
    
SignalChangedEvent - [device ID][Device Name][room Name][signal event ID][signal Value] - Signal event ID differs by device and signal value is either bool or int based on the eventID.

SignalChangedEventWithStrings - [device ID][Device Name][Signal Name][Signal Value string][Signal direction][message]


In [1]:
import os 
import pandas as pd
import numpy as np
import h5py
import json
from numpy import nan
from datetime import datetime, timedelta, date
from fractions import Fraction
from time import mktime
import time
import requests
import os.path
pd.set_option('display.max_columns', 500)

/Users/armanrahman/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Function to quickly load or save dataframes as h5 files

In [2]:
def save_or_load_from_checkpoint(checkpoint_name):
    if os.path.isfile(checkpoint_name):
        return pd.read_hdf(checkpoint_name,'table')
    df.to_hdf(checkpoint_name, 'table', mode='w', append=True, complevel=9, complib='zlib', index=False)
    return df

### Loading logs and initial cleaning

First we read in the data from the original .BAC files, do a basic first pass of formatting the data, and set up a large  dataframe to hold the data for each event type. We start by separating the log level and time stamp since they are not pipe separeated, and then split the message into columns the correspond to the the log fields and the EventType dependent strings

In [ ]:
# column labels
labels = ['LogLevel',
          'TimeStamp',
          'LogVisibility',
          'LogSeverity',
          'entryType',
          'entrySubType',
          'eventType']
# from label list
EVENT_TYPE_INDEX = labels.index('eventType')

# from line in .bac file
LOG_LEVEL_START = 0
LOG_LEVEL_END = 3
TIMESTAMP_START = 6
TIMESTAMP_END = 14
PIPE_SEPARATED_DATA_START = 17

event_type_labels = [
    "string message", # GeneralMessage
    "load ID", "loadName", "roomName1", "rampTime", "rampBaseValue", "rampFinalValue", # LevelChangedEvent
    "keypad ID", "keypadName", "roomName2", "buttonNum", "buttonState", # ButtonChangedEvent
    "signalID", "signalName", "roomName3", "RemoteSystemEvent string", # RemoteSystemEvent
    "ID", "name", "roomName4", "message1", # TimeClockChangedEvent/OccupancyChangedEvent/SceneChangedEvent
    "device ID1", "Name", "roomName5", "connection status", "Load 1 Room Name:Load 1 Name", "Load 2 Room Name:Load 2 Name", # ConnectionStatus/DeviceConnectionStatusWithOptions
    "device ID2", "Device Name1", "roomName6", "signal event ID", "signal Value", # SignalChangedEvent
    "device ID3", "Device Name2", "Signal Name", "Signal Value string", "Signal direction", "message2", # SignalChangedEventWithStrings
]

def from_bac():
    # device_id_offsets = [i for i, x in enumerate(event_type_labels) if x == "device ID"]
    clean_lines = []

    for log in os.listdir('data'):
        with open('data/{}'.format(log)) as logfile:
            for line in logfile:
                line = line.rstrip('\n')
                if line[-1] == '|':
                    line = line[:-1]
                all_data = ([line[LOG_LEVEL_START:LOG_LEVEL_END], line[TIMESTAMP_START:TIMESTAMP_END]]
                            + line[PIPE_SEPARATED_DATA_START:].split('|'))

                event_type_dependent_strings = all_data[len(labels):]
                clean_line = all_data[:len(labels)]

                START_INDEX = 7
                if clean_line[EVENT_TYPE_INDEX] == 'GeneralMessage':
                    START_INDEX += event_type_labels.index('string message')
                    # account for pipes in the message string
                    event_type_dependent_strings = ['|'.join(event_type_dependent_strings)]
                    assert START_INDEX == 7
                    assert len(event_type_dependent_strings) == 1, event_type_dependent_strings

                elif clean_line[EVENT_TYPE_INDEX] == 'LevelChangedEvent':
                    START_INDEX += event_type_labels.index('load ID')
                    assert START_INDEX == 8
                    assert len(event_type_dependent_strings) == 6

                elif clean_line[EVENT_TYPE_INDEX] == 'ButtonChangedEvent':
                    START_INDEX += event_type_labels.index('keypad ID')
                    assert START_INDEX == 14
                    assert len(event_type_dependent_strings) == 5

                elif clean_line[EVENT_TYPE_INDEX] == 'RemoteSystemEvent':
                    START_INDEX += event_type_labels.index('signalID')
                    assert START_INDEX == 19
                    assert len(event_type_dependent_strings) == 4

                elif (clean_line[EVENT_TYPE_INDEX] == 'TimeClockChangedEvent' or
                      clean_line[EVENT_TYPE_INDEX] == 'OccupancyChangedEvent' or
                      clean_line[EVENT_TYPE_INDEX] == 'SceneChangedEvent'):
                    START_INDEX += event_type_labels.index('ID')
                    assert START_INDEX == 23
                    assert len(event_type_dependent_strings) == 4

                elif (clean_line[EVENT_TYPE_INDEX] == 'ConnectionStatus' or
                      clean_line[EVENT_TYPE_INDEX] == 'DeviceConnectionStatusWithOptions'):
                    START_INDEX += event_type_labels.index('device ID1')
                    assert START_INDEX == 27
                    assert (len(event_type_dependent_strings) == 4 or
                            len(event_type_dependent_strings) == 5 or
                            len(event_type_dependent_strings) == 6)

                elif clean_line[EVENT_TYPE_INDEX] == 'SignalChangedEvent':
                    START_INDEX += event_type_labels.index('device ID2')
                    assert START_INDEX == 33
                    assert len(event_type_dependent_strings) == 5

                elif clean_line[EVENT_TYPE_INDEX] == 'SignalChangedEventWithStrings':
                    START_INDEX += event_type_labels.index('device ID3')
                    assert START_INDEX == 38
                    # to correct for the double pipe in "Basement Mudroom"
                    event_type_dependent_strings = [i for i in event_type_dependent_strings if i]
                else:
                    raise ValueError("Wrong event type: {}".format(clean_line[EVENT_TYPE_INDEX]))

                clean_line = clean_line + [np.nan]*len(event_type_labels) + [log[-14:-4]]
                clean_line[START_INDEX:START_INDEX + len(event_type_dependent_strings)] = event_type_dependent_strings
                clean_lines.append(clean_line)
    df = pd.DataFrame(clean_lines, columns=labels + event_type_labels + ["date"])
    return df

## Load data from BAC files

In [ ]:
df = from_bac()

### Combine date and time

In [ ]:
def to_datetime(row):
    new_date = row['date'] + ' ' + row['TimeStamp']
    dt = datetime.strptime(new_date, '%Y-%m-%d %H:%M:%S')
    unix_secs = mktime(dt.timetuple())
    return unix_secs

In [ ]:
df['unix_time'] = df.apply(to_datetime, axis=1)
df['unix_time'] = pd.to_datetime(df['unix_time'], unit='s')
df = df.sort_values(by='unix_time')
df = df.drop(["TimeStamp","date"], axis=1)

### Drop columns that don't provide additional information that the algorithms can use

In [ ]:
# Unnecessary general columns
df = df.drop(["LogLevel", "LogVisibility", "LogSeverity"],axis=1, errors='ignore')
 
# Unnecessary after deleting General Message
df = df.drop("string message", axis=1, errors='ignore')
 
# Unnecessary after deleting Button Change Event
df = df.drop(["keypad ID", "keypadName", "roomName2", "buttonNum", "buttonState"], axis=1, errors='ignore')
 
# Unnecessary after deleting Connection Status
df = df.drop(["device ID1", "Name", "roomName5", "connection status", "Load 1 Room Name:Load 1 Name", "Load 2 Room Name:Load 2 Name"], axis=1, errors='ignore')
 
# Unnecessary after deleting Remote System Event
df = df.drop(["signalID", "signalName", "roomName3", "RemoteSystemEvent string"], axis=1, errors='ignore')

# Unnecessary because redundant with Signal Name
df = df.drop(["Device Name2"], axis=1, errors='ignore')

### Remove event types that don't provide additional information that we can use

In [ ]:
df = df[df['eventType'] != 'ButtonChangedEvent']
df = df[df['eventType'] != 'GeneralMessage']
df = df[df['eventType'] != 'RemoteSystemEvent']
df = df[df['eventType'] != 'TimeClockChangedEvent']
df = df[df['eventType'] != 'ConnectionStatus']
df = df[df['eventType'] != 'DeviceConnectionStatusWithOptions']
df = df[df['eventType'] != 'SignalChangedEvent']

### Merge Ids and Names

We combined all of the EventType dependent string fields that seemed to refer to the same thing in order to remore redundant columns

In [ ]:
# Merge room names
df['room_name_merged'] = df['roomName1'].fillna('') + df['roomName4'].fillna('') + df['roomName6'].fillna('') + df['Signal Name'].fillna('')
df = df.drop(["roomName1","roomName4","roomName6", "Signal Name"], axis=1)

In [ ]:
# Merge inner room locations
df['location_in_room'] = df['loadName'].fillna('') + df['Device Name1'].fillna('')
df = df.drop(["loadName","Device Name1"], axis=1)

In [ ]:
# Merge IDs 
df['device_id'] = df['load ID'].fillna('') + df['ID'].fillna('') + df['device ID2'].fillna('') + df['device ID3'].fillna('')
df = df.drop(['load ID', 'ID', 'device ID2', 'device ID3'], axis=1)

In [ ]:
# Merge messages 
df['message_merged'] = df['message1'].fillna('') + df['message2'].fillna('')
df = df.drop(["message1", "message2"], axis=1)

In [ ]:
df = df.replace('', np.NaN)

Try to find unique combination for true ID's

In [ ]:
for device_id in df['device_id'].unique():
    print("device_id: " + str(device_id) + " unique in other field: " + str(len(df[df['device_id'] == device_id]['eventType'].unique())))

### Combine room name, id, and entry type for truly unique ids

The devices had no unique identifier stored. We created unique identifiers for each device by combining room name, id, and entry type.

In [ ]:
df['room_name_merged'] = df['room_name_merged'].str.lower().str.replace(' ', '-')
df['entryType'] = df['entryType'].str.lower()

In [ ]:
df['device_entry_id'] = df['device_id'].astype(str) + '_' + df['room_name_merged'].astype(str) + '_' + df['entryType'].astype(str)
df = df.drop(["device_id"], axis=1)

In [ ]:
df = df.drop_duplicates()

In [ ]:
# Checkpoint 
df = save_or_load_from_checkpoint('./checkpoints/data_0.h5')  

### Drop unneccesary rows

In [ ]:
# dropping more useless data
index_to_drop = df.loc[df["entryType"] == "Auxiliary"][df['eventType'] == "SignalChangedEventWithStrings"].index.tolist()
df = df.drop(index_to_drop)

In [ ]:
# dropping repeated data
index_to_drop = df.loc[((df['entryType'] == "System") | (df['entryType'] == "DoorLock"))& (df["name"].notnull())].index.tolist()
df = df.drop(index_to_drop)

### Get temperature values from 'Signal direction'

In [ ]:
def get_setpoints(row):
    signal_direction = str(row['Signal direction'])
    temp = np.nan
    if 'Fahrenheit' in signal_direction and signal_direction[0].isdigit():
        temp = int(''.join(x for x in signal_direction if x.isdigit()))
    return temp
    

In [ ]:
df['temperature'] = df.apply(get_setpoints, axis=1)

In [ ]:
df = save_or_load_from_checkpoint('./checkpoints/data_1.h5')  

## Get Event Data

We looked through the different SignalChangedEventWithStrings messages that have relevant data and picked out the ones that refer to categorical events and stored the values in a separate column named 'event'

In [ ]:
def check_signal_direction(df, string_val):
    return df[df["Signal Value string"] == string_val]['Signal direction'].unique().astype(str)

In [ ]:
d = {}
for i in sorted(df["Signal Value string"].astype(str).unique()):
    d[i] = check_signal_direction(df, i)
for key, value in d.items():
    print(key,value)

After some manual inspection this is the dictionary that we settled on that contains events which are in a format that we can use

In [ ]:
d = {
    'Mode': ['Heat'
             'Cool'
             'Off'],
    'Auto Mode': ['Enabled'
                  'Disabled'],
    'Single Setpoint Mode': ['Disabled'],
    'Slab 5B': ['Inactive'],
    'Humidity View': ['Enabled'],
    'Slab 2': ['Inactive'],
    'Heat Stage 1': ['Active'
                     'Inactive'],
    'Cool Stage 1': ['Inactive'
                     'Active'],
    'Active Fan Level': ['Off'
                         'High'],
    'Fan': ['On'
            'Auto'],
    'Hold': ['On'
             'Off'],
    'Humidifier Enable': ['Inactive'],
    'Slab 4B': ['Inactive'],
    'Slab 1': ['Inactive'],
    'Slab 3': ['Inactive'],
    'Humidity Mode': ['Enabled'],
    'Floor Warming': ['Heat:False'
                      'Off:False'
                      'Off:True'],
    'Cool Mode': ['Enabled'],
    'Humidifier Off': ['Inactive'
                       'Active'],
    'Slab 4A': ['Inactive'],
    'Heat Mode': ['Enabled'],
    'Slab 5A': ['Inactive']
}

This function is used to encode the events that have their values in the 'Signal Direction' <br> *This seems like this could be a bug? Are these messages supposed to be in the message column instead?

In [ ]:
def encode_event(row, **kwargs):
    """kwargs = [Signal Value, values_dict]"""
    if str(row['eventType']) == 'SignalChangedEventWithStrings':
#         import pdb; pdb.set_trace()
        for key, values in kwargs.items():
            if str(row['Signal Value string']) == key:
                for val in values:
                    if str(row['Signal direction']) == val:
                        return key + val
    return np.nan

In [ ]:
df['event'] = df.apply(encode_event, **d, axis=1)

This function performs similarly to the one above, but the values for the lock/unlock and occupancy/vacancy events are in a different column from the rest of the data

In [ ]:
def is_locked_or_occupied(row):
    if str(row['eventType']) == 'OccupancyChangedEvent':
        return row['message_merged']
    elif str(row['Signal Value string']) == 'Lock' or str(row['Signal Value string']) == 'Unlock':
        return row['entryType'] + row['Signal Value string']
    elif pd.notna(row['event']):
        return str(row['event'])
    return np.nan

In [ ]:
df['event'] = df.apply(is_locked_or_occupied, axis=1)

## Get Regression Data

In [ ]:
def get_value(row):
    value = np.nan
    if pd.notna(row['rampFinalValue']):
        if int(row['rampTime']) == 0:
            value = int(row['rampFinalValue'])
    elif pd.notna(row['temperature']):
        value = int(row['temperature'])
    elif pd.notna(row['signal Value']):
        value = int(row['signal Value'])
    elif str(row['entryType']) == 'Shades' and str(row['eventType']) == 'SceneChangedEvent':
        base_str = str(row['name'])
        split_str = base_str.split(' ')
        first_str = split_str[0]
        is_closed = True if split_str[1] == 'Closed' else False
        if first_str[-1].isdigit():
            frac = float(Fraction(first_str))
            if is_closed:
                frac = 1 - frac
            value = frac
        elif is_closed:
            value = 0
        else:
            value = 1
    return value

In [ ]:
df['value'] = df.apply(get_value, axis=1)

In [ ]:
def get_event_type(row): 
    event_type = np.nan
    if pd.notna(row['value']):
        event_type = str(row['entryType'])
    return event_type

In [ ]:
df['regression_value_type'] = df.apply(get_event_type, axis=1)

In [ ]:
df = save_or_load_from_checkpoint('./checkpoints/data_2.h5')  

## Extract final dataframe

Create a new dataframe containing only the columns that we will be using for statistical analysis and ML exploration## Extract final dataframe

In [ ]:
df = df[['device_entry_id', 'room_name_merged', 'location_in_room', 'event', 'regression_value_type', 'value', 'unix_time']]

#### Convert to GMT

In [ ]:
df['unix_time'] = df['unix_time'] + pd.DateOffset(hours=4)

#### Remove null rows

In [ ]:
df = df.loc[(df['event'].notnull()) | (df['regression_value_type'].notnull()) | (df['value'].notnull())]

### Add week, day, month, hour

In [ ]:
def add_dates(data_frame):
    date = pd.to_datetime(data_frame['unix_time'], unit='s')
    data_frame['week'] = date.dt.week
    data_frame['day'] = date.dt.day
    data_frame['month'] = date.dt.month
    data_frame['hour'] = date.dt.hour
    data_frame['minute'] = date.dt.minute
    data_frame['second'] = date.dt.second
    return data_frame

# df = add_dates(df)
# df = df.drop('unix_time', axis=1)

### Add temperature, sunrise/sunset data

Here we use the darksky weather API to store data about the weather on each given day in a local temperature_data.json file.

In [ ]:
def create_temp_time_dict(data_frame):
    base_http = 'https://api.darksky.net/forecast/afeffdaf32e862d1c6d7279c7f5df74f/39.833851,-74.871826,'
    end_http = '?exclude=currently,flags,alerts,minutely'
    hourly_temp_dict = {}
    
    dates = data_frame['unix_time'].map(pd.Timestamp.date).unique()
    
    for date in dates:
        unix_date = int(time.mktime(date.timetuple()))
        response = requests.get(base_http + str(unix_date) + end_http) 
        temp_json = response.json()
        sunset_time = temp_json['daily']['data'][0]['sunsetTime']
        sunrise_time = temp_json['daily']['data'][0]['sunriseTime']
        for hour_data in temp_json['hourly']['data']:
            hourly_temp_dict[hour_data['time']] = hour_data
            hourly_temp_dict[hour_data['time']]['sunset_time'] = sunset_time
            hourly_temp_dict[hour_data['time']]['sunrise_time'] = sunrise_time
        
    with open('temperature_data_hourly.json', 'w') as outfile:
        json.dump(hourly_temp_dict, outfile)


In [ ]:
json_data = None
if not os.path.isfile('temperature_data_hourly.json'):
    create_temp_time_dict(df)
with open('temperature_data_hourly.json') as f:
    json_data = json.load(f)

From the json object we just created, we pull the hourly temperature, and determine whether the sun was up or down 
for each row in the data. We add two rows to the dataframe, 'sun', and 'outside_temperature'. 

In [ ]:
def create_weather_sunset_columns(row):
    #hard-coded year for now. 
    date = row['unix_time'].date()
    hour = row['unix_time'].hour
    minute = row['unix_time'].minute
    
    #format for API
    date_timestamp = int(time.mktime(date.timetuple()))
    hour_timestamp = date_timestamp + hour * 60 * 60
    minute_timestamp = hour_timestamp + minute * 60
    
    hour_timestamp_string = str(hour_timestamp)
    
    temperature_info = json_data[hour_timestamp_string]['temperature']
    sunrise = json_data[hour_timestamp_string]['sunrise_time'] #in GMT 
    sunset = json_data[hour_timestamp_string]['sunset_time']

    is_sun_up = 1 if sunrise <= minute_timestamp <= sunset else 0
    return is_sun_up, temperature_info

In [ ]:
df['sun'], df['outside_temperature'] = zip(*df.apply(create_weather_sunset_columns, axis=1))

In [ ]:
df.head()

In [3]:
df = save_or_load_from_checkpoint('./checkpoints/data_3.h5')  

In [42]:
sorted(df.device_entry_id.unique())

['100Master BathLights',
 '104Master BedShades',
 '105Master BedShades',
 '106First Floor MudroomShades',
 '106Master Walk In ClosetLights',
 '108FoyerShades',
 '113DinetteShades',
 '115Office First FloorShades',
 '116Guest RoomLights',
 '116Office First FloorShades',
 '118Upstairs LandingLights',
 '11Basement BathroomLights',
 '120Upstairs LandingLights',
 '122Upstairs LandingLights',
 '133Office Second FloorLights',
 '143KitchenLights',
 '146Back YardLights',
 '159TheaterLights',
 '15Basement BathroomLights',
 '161TheaterLights',
 '164TheaterLights',
 '17Basement Rec RoomLights',
 '183Madis BedroomShades',
 "184Alex's bedroomShades",
 '185Master BedShades',
 "186Alex's bedroomShades",
 '187Master BedShades',
 '188Master BedShades',
 '189Master BedShades',
 '190Guest RoomShades',
 '191Master BedShades',
 '192KitchenClimate',
 '192UnassignedClimate',
 '194Master BedClimate',
 '194UnassignedClimate',
 '195Great RoomClimate',
 '195UnassignedClimate',
 '196Madis BedroomShades',
 '199Foyer

## Get event only data (?)

In [31]:
df.groupby('device_entry_id').count().sort_values(by='room_name_merged', ascending=False)

,room_name_merged,location_in_room,event,regression_value_type,value,unix_time,sun,outside_temperature
device_entry_id,,,,,,,,
194Master BedClimate,19079,0,63,19016,19016,19079,19079,19079
195Great RoomClimate,9381,0,63,9318,9318,9381,9381,9381
192KitchenClimate,9125,0,105,9020,9020,9125,9125,9125
26Master BathAuxiliary,5619,0,5619,0,0,5619,5619,5619
30Second Floor BathroomAuxiliary,4682,0,4682,0,0,4682,4682,4682
1Basement Rec RoomAuxiliary,4043,0,4043,0,0,4043,4043,4043
29Guest RoomAuxiliary,3509,0,3509,0,0,3509,3509,3509
21GarageAuxiliary,2621,0,2621,0,0,2621,2621,2621
17Basement Rec RoomLights,2392,2392,0,2392,2392,2392,2392,2392


In [ ]:
['194Master BedClimate', '118Upstairs LandingLights', '195Great RoomClimate', '29Basement StairsLights', '86KitchenLights']

In [5]:
min_time = df['unix_time'].min() - pd.Timedelta(seconds=df['unix_time'].min().second)
max_time = df['unix_time'].max() - pd.Timedelta(seconds=df['unix_time'].max().second)
date_range = pd.date_range(min_time, max_time, freq="1min")
state_df = pd.DataFrame(index=date_range, columns=df['device_entry_id'].unique())

In [6]:
state_df.head()

,118Upstairs LandingLights,208Basement MudroomDoorLock,194Master BedClimate,30Second Floor BathroomAuxiliary,26Master BathAuxiliary,90KitchenLights,92KitchenLights,88KitchenLights,94KitchenLights,50First Floor HallwayLights,54First Floor HallwayLights,86KitchenLights,48First Floor HallwayLights,56Front YardLights,195Great RoomClimate,192KitchenClimate,104Master BedShades,189Master BedShades,185Master BedShades,191Master BedShades,187Master BedShades,188Master BedShades,1Basement Rec RoomAuxiliary,17Basement Rec RoomLights,21GarageAuxiliary,29Basement StairsLights,29Guest RoomAuxiliary,183Madis BedroomShades,196Madis BedroomShades,23Basement Rec RoomLights,19Basement Rec RoomLights,21Basement Rec RoomLights,146Back YardLights,2Back YardLights,5Basement MudroomLights,7Basement MudroomLights,11Basement BathroomLights,15Basement BathroomLights,105Master BedShades,207Office First FloorShades,64Dining RoomLights,43First Floor BathroomLights,82Great RoomLights,77Great RoomLights,79Great RoomLights,84Great RoomLights,164TheaterLights,161TheaterLights,159TheaterLights,100Master BathLights,75Family RoomLights,73Family RoomLights,27Basement Rec RoomLights,194UnassignedClimate,195UnassignedClimate,192UnassignedClimate,184Alex's bedroomShades,186Alex's bedroomShades,190Guest RoomShades,206DinetteShades,199FoyerShades,201First Floor MudroomShades,200First Floor MudroomShades,66FoyerLights,71FoyerLights,9Basement MudroomLights,93Whole HouseShades,92Whole HouseShades,45First Floor BathroomLights,52LibraryLights,34KitchenLights,62First Floor MudroomLights,58First Floor MudroomLights,120Upstairs LandingLights,39Office First FloorLights,133Office Second FloorLights,106Master Walk In ClosetLights,96Master BedShades,97Master BedShades,122Upstairs LandingLights,217Office First FloorLights,116Guest RoomLights,113DinetteShades,106First Floor MudroomShades,108FoyerShades,115Office First FloorShades,116Office First FloorShades,143KitchenLights,35Billiards RoomLights,49First Floor HallwayLights,37Billiards RoomLights
2017-08-04 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-04 08:01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-04 08:02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-04 08:03:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-04 08:04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
time_index = df.iloc[0]['unix_time'] - pd.Timedelta(seconds=df.iloc[0]['unix_time'].second)
time_index

Timestamp('2017-08-04 08:00:00')

In [13]:
state_df.loc[time_index]

118Upstairs LandingLights           NaN
208Basement MudroomDoorLock         NaN
194Master BedClimate                NaN
30Second Floor BathroomAuxiliary    NaN
26Master BathAuxiliary              NaN
90KitchenLights                     NaN
92KitchenLights                     NaN
88KitchenLights                     NaN
94KitchenLights                     NaN
50First Floor HallwayLights         NaN
54First Floor HallwayLights         NaN
86KitchenLights                     NaN
48First Floor HallwayLights         NaN
56Front YardLights                  NaN
195Great RoomClimate                NaN
192KitchenClimate                   NaN
104Master BedShades                 NaN
189Master BedShades                 NaN
185Master BedShades                 NaN
191Master BedShades                 NaN
187Master BedShades                 NaN
188Master BedShades                 NaN
1Basement Rec RoomAuxiliary         NaN
17Basement Rec RoomLights           NaN
21GarageAuxiliary                   NaN


In [16]:
def create_state(row, *args):
#     import pdb; pdb.set_trace()
    device_id = str(row['device_entry_id'])
    time_index = row['unix_time'] - pd.Timedelta(seconds=row['unix_time'].second)
    new_df = args[0]
    if pd.notna(row['value']):
        new_df.loc[time_index][device_id] = int(row['value'])
    else:
        new_df.loc[time_index][device_id] = str(row['event'])
    

In [17]:
df.apply(create_state, args=[state_df], axis=1)

5203      None
5195      None
5211      None
5214      None
5239      None
5240      None
5242      None
5253      None
5325      None
5328      None
5334      None
5364      None
5365      None
5449      None
5451      None
5459      None
5450      None
5434      None
5435      None
5436      None
5433      None
5465      None
5507      None
5535      None
5552      None
5550      None
5551      None
5554      None
5556      None
5636      None
          ... 
504917    None
504944    None
504946    None
504950    None
504954    None
504962    None
504965    None
504993    None
504995    None
505012    None
505068    None
505102    None
505130    None
505200    None
505205    None
505232    None
505240    None
505242    None
505246    None
505252    None
505276    None
505278    None
505281    None
505285    None
505288    None
505289    None
505292    None
505293    None
505296    None
505300    None
Length: 72566, dtype: object

In [18]:
state_df.head(20)

,118Upstairs LandingLights,208Basement MudroomDoorLock,194Master BedClimate,30Second Floor BathroomAuxiliary,26Master BathAuxiliary,90KitchenLights,92KitchenLights,88KitchenLights,94KitchenLights,50First Floor HallwayLights,54First Floor HallwayLights,86KitchenLights,48First Floor HallwayLights,56Front YardLights,195Great RoomClimate,192KitchenClimate,104Master BedShades,189Master BedShades,185Master BedShades,191Master BedShades,187Master BedShades,188Master BedShades,1Basement Rec RoomAuxiliary,17Basement Rec RoomLights,21GarageAuxiliary,29Basement StairsLights,29Guest RoomAuxiliary,183Madis BedroomShades,196Madis BedroomShades,23Basement Rec RoomLights,19Basement Rec RoomLights,21Basement Rec RoomLights,146Back YardLights,2Back YardLights,5Basement MudroomLights,7Basement MudroomLights,11Basement BathroomLights,15Basement BathroomLights,105Master BedShades,207Office First FloorShades,64Dining RoomLights,43First Floor BathroomLights,82Great RoomLights,77Great RoomLights,79Great RoomLights,84Great RoomLights,164TheaterLights,161TheaterLights,159TheaterLights,100Master BathLights,75Family RoomLights,73Family RoomLights,27Basement Rec RoomLights,194UnassignedClimate,195UnassignedClimate,192UnassignedClimate,184Alex's bedroomShades,186Alex's bedroomShades,190Guest RoomShades,206DinetteShades,199FoyerShades,201First Floor MudroomShades,200First Floor MudroomShades,66FoyerLights,71FoyerLights,9Basement MudroomLights,93Whole HouseShades,92Whole HouseShades,45First Floor BathroomLights,52LibraryLights,34KitchenLights,62First Floor MudroomLights,58First Floor MudroomLights,120Upstairs LandingLights,39Office First FloorLights,133Office Second FloorLights,106Master Walk In ClosetLights,96Master BedShades,97Master BedShades,122Upstairs LandingLights,217Office First FloorLights,116Guest RoomLights,113DinetteShades,106First Floor MudroomShades,108FoyerShades,115Office First FloorShades,116Office First FloorShades,143KitchenLights,35Billiards RoomLights,49First Floor HallwayLights,37Billiards RoomLights
2017-08-04 08:00:00,19275,DoorLockLock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-04 08:01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-04 08:02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-04 08:03:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-04 08:04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-04 08:05:00,NaN,NaN,NaN,NaN,NaN,NaN

In [20]:
state_df.fillna(method='ffill').head(20)

,118Upstairs LandingLights,208Basement MudroomDoorLock,194Master BedClimate,30Second Floor BathroomAuxiliary,26Master BathAuxiliary,90KitchenLights,92KitchenLights,88KitchenLights,94KitchenLights,50First Floor HallwayLights,54First Floor HallwayLights,86KitchenLights,48First Floor HallwayLights,56Front YardLights,195Great RoomClimate,192KitchenClimate,104Master BedShades,189Master BedShades,185Master BedShades,191Master BedShades,187Master BedShades,188Master BedShades,1Basement Rec RoomAuxiliary,17Basement Rec RoomLights,21GarageAuxiliary,29Basement StairsLights,29Guest RoomAuxiliary,183Madis BedroomShades,196Madis BedroomShades,23Basement Rec RoomLights,19Basement Rec RoomLights,21Basement Rec RoomLights,146Back YardLights,2Back YardLights,5Basement MudroomLights,7Basement MudroomLights,11Basement BathroomLights,15Basement BathroomLights,105Master BedShades,207Office First FloorShades,64Dining RoomLights,43First Floor BathroomLights,82Great RoomLights,77Great RoomLights,79Great RoomLights,84Great RoomLights,164TheaterLights,161TheaterLights,159TheaterLights,100Master BathLights,75Family RoomLights,73Family RoomLights,27Basement Rec RoomLights,194UnassignedClimate,195UnassignedClimate,192UnassignedClimate,184Alex's bedroomShades,186Alex's bedroomShades,190Guest RoomShades,206DinetteShades,199FoyerShades,201First Floor MudroomShades,200First Floor MudroomShades,66FoyerLights,71FoyerLights,9Basement MudroomLights,93Whole HouseShades,92Whole HouseShades,45First Floor BathroomLights,52LibraryLights,34KitchenLights,62First Floor MudroomLights,58First Floor MudroomLights,120Upstairs LandingLights,39Office First FloorLights,133Office Second FloorLights,106Master Walk In ClosetLights,96Master BedShades,97Master BedShades,122Upstairs LandingLights,217Office First FloorLights,116Guest RoomLights,113DinetteShades,106First Floor MudroomShades,108FoyerShades,115Office First FloorShades,116Office First FloorShades,143KitchenLights,35Billiards RoomLights,49First Floor HallwayLights,37Billiards RoomLights
2017-08-04 08:00:00,19275,DoorLockLock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-04 08:01:00,19275,DoorLockLock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-04 08:02:00,19275,DoorLockLock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-04 08:03:00,19275,DoorLockLock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-04 08:04:00,19275,DoorLockLock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [40]:
test_state_df = state_df[['194Master BedClimate', '118Upstairs LandingLights', '195Great RoomClimate', '29Basement StairsLights', '86KitchenLights']]
test_state_df = test_state_df.fillna(method='ffill')

In [38]:
test_state_df.dtypes

194Master BedClimate          object
118Upstairs LandingLights      int64
195Great RoomClimate          object
29Basement StairsLights      float64
86KitchenLights              float64
dtype: object

### Encode Columns

Convert Categorical to Numerical Data. 


In [ ]:
from sklearn import preprocessing
columns_to_update = ["device_id_name", "event", "regression_value_type"]

def encode_columns(data_frame, column_names):
    label_encoders = {}
    for col in column_names:
        values = data_frame[col].unique()
        le = preprocessing.LabelEncoder()
        le.fit(values)
        label_encoders[col] = le
        numerical_values = le.transform(data_frame[col])
        data_frame[col] = numerical_values
    return data_frame, label_encoders

categorical_df = df.copy()
# to encode the df we need to encode the NaNs as strings
categorical_df = categorical_df.fillna({'regression_value_type':'nan', 'event': 'nan', 'value': 0})
categorical_df, label_encoders = encode_columns(categorical_df, columns_to_update)

In [ ]:
categorical_df.to_hdf('./checkpoints/data_categorical.h5', 'table', mode='w', append=True, complevel=9, complib='zlib', index=False)

### One-hot encode columns


starting with binary

In [ ]:
def one_hot_encode_col(data_frame, col_name):
    one_hot_encoded = pd.get_dummies(data_frame[col_name], prefix=col_name)
    return one_hot_encoded

In [ ]:
binary_df = df.copy()
binary_df = binary_df.drop('regression_value_type', axis=1)
binary_df = binary_df.drop('value', axis=1)

In [ ]:
event_encoded = one_hot_encode_col(binary_df, 'event')
binary_df = pd.concat([binary_df, event_encoded], axis=1)
binary_df = binary_df.drop('event', axis=1)

In [ ]:
binary_df.to_hdf('./checkpoints/data_binary_encoded.h5', 'table', mode='w', append=True, complevel=9, complib='zlib', index=False)